In [7]:
import clusim.sim as csim
import leidenalg
from igraph import Graph
from clusim.clustering import Clustering, print_clustering
import pandas as pd
from infomap import Infomap
import numpy as np
import xnetwork as xn
import graph_tool as gt
import graph_tool.inference as gtInference
from graph_tool import Graph as gtGraph
from pathlib import Path 
from tqdm.auto import tqdm
from multiprocessing import Pool
from functools import partial
from sklearn.decomposition import PCA
from collections import defaultdict
from functools import partial

In [ ]:
data_demo=pd.read_csv('/Users/fsfatemi/local_AD/KBASE_datashare/demo_final2024.merge_phenotype.506.csv')
data_demo.head()

In [ ]:
data_GlCtx_CL_log=data_demo[['Barcode','GlCtx_CL_log']]
data_GlCtx_CL_log.set_index('Barcode', inplace=True)
data_GlCtx_CL_log= data_GlCtx_CL_log.T
data_GlCtx_CL_log.head()

In [ ]:
data_deposition=pd.read_csv('/Users/fsfatemi/local_AD/KBASE_datashare/residuals_cqn_cpm.13603_1140.csv')
data_deposition.set_index('Barcode', inplace=True)
data_deposition=data_deposition.T
data_deposition.head()

In [71]:
def calculate_eigengene(values_df):


    beta_values_matrix = values_df.values
    pca = PCA(n_components=1)
    pca.fit(beta_values_matrix)
    eigengene_values = pca.components_
    return pd.DataFrame(eigengene_values,columns=data_deposition.columns)

In [82]:
def calculateCorrelations(cg_names):

    data_module=data_deposition.loc[data_deposition.index.isin(cg_names)]
    eigengene_df = calculate_eigengene(data_module)

    series1 = eigengene_df.iloc[0]
    series2 = data_GlCtx_CL_log[eigengene_df.columns].iloc[0]
    correlation = series1.corr(series2)
    
    return {"correlation": correlation}

In [8]:
def membership_to_clusters(membership_vector):

    clusters = defaultdict(set)

    for idx, cluster_id in enumerate(membership_vector):
        clusters[cluster_id].add(idx)
    

    cluster_list = [set(cluster) for cluster in clusters.values()]
    
    return cluster_list

In [78]:
resolutions = [0.001,0.005,0.01,0.1,1.0,0.5,5.0,10.0,20.0,50,100]
markovTimes = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,5,10]

def processnetwork(networkFile):

    g = xn.load(networkFile)
    entries = []

    for res in resolutions:

        #leiden weighted
        propertyName="Leiden_weighted_%f"%res
        membership_list=g.vs[propertyName]

        clusters = membership_to_clusters(membership_list)

        for c in clusters:
            cg_names=[g.vs[idx]['Label'] for idx in c]
            entries.append({"cluster": cg_names,
                            "cluster size":len(cg_names),
                            "resolution":res,
                            "network":networkFile.stem,
                            "method":'leiden_weighted',
                            })

            entries[-1].update(calculateCorrelations(cg_names))
        
        
        
        #leiden unweighted
        propertyName="Leiden_unweighted_%f"%res
        membership_list=g.vs[propertyName]

        clusters = membership_to_clusters(membership_list)

        for c in clusters:
            
            cg_names=[g.vs[idx]['Label'] for idx in c]
            entries.append({"cluster": cg_names,
                            "cluster size":len(cg_names),
                            "resolution":res,
                            "network":networkFile.stem,
                            "method":'leiden_unweighted',
                            })

            entries[-1].update(calculateCorrelations(cg_names))
    del g
    return entries

In [ ]:


if __name__ == "__main__":
    networksWithCommunitiesPath = Path("KNN_NetworksWithCommunities")
    networkFiles = list(networksWithCommunitiesPath.glob("*.xnet"))
    print(networkFiles)
    allEntries = []
    for networkFile in networkFiles:
        entries = processnetwork(networkFile)
        allEntries += entries
    
    df = pd.DataFrame(allEntries)
    df.to_csv("Correlation_results/module_correlations_leiden_all.csv", index=False)

